# YOLOv5 Custom Training 🚀

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Requirements

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 9), reused 23 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 28.08 MiB/s, done.
Resolving deltas: 100% (10821/10821), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
Setup complete. Using torch 2.0.1+cu118 (Tesla T4)


## Roboflow Dataset


In [ ]:
!pip install roboflow>=1.0.1
from roboflow import Roboflow
rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3",model_format="yolov5", notebook="ultralytics")

In [ ]:
!mkdir /content/datasets
!cd /content/datasets

mkdir: cannot create directory ‘/content/datasets’: File exists


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3")
project = rf.workspace("sappia").project("hri-ra-mtc9l")
dataset = project.version(3).download("yolov5")


%cd /content/yolov5

## Train ⚡

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset location is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 640 --batch 32 --epochs 20 --data /content/datasets/HRI-RA-3/data.yaml --weights yolov5m.pt --cache

train: weights=yolov5m.pt, cfg=, data=/content/datasets/HRI-RA-3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-193-g485da42 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015

## Evaluation 📏

### Test normally


> *FP16 half-precision can be used during inference by adding the **--half** flag!*

In [ ]:
!python val.py --weights /content/drive/MyDrive/EAI/HRI_RA/GarbageDet/best.pt --data /content/datasets/HRI-RA-3/data.yaml --img 640 --half

val: data=/content/datasets/HRI-RA-3/data.yaml, weights=['/content/drive/MyDrive/EAI/HRI_RA/GarbageDet/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-193-g485da42 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/datasets/HRI-RA-3/valid/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:03<00:00,  1.61s/it]
                   all         50         50      0.981      0.956      0.995      0.797
               compost         50         35          1      0.952      0.995    

### Post Pruning ✂

We repeat the above test with a pruned model by using the *torch_utils.prune()* command. We create an update version of *val.py* file to prune the model.

In [ ]:
!mv val.py val_old.py
!cp val_old.py val.py
!mv val.py val_prune.py
!mv val_old.py val.py

Add this lines of code to the new created *val_prune.py* file:
<a href="https://imgur.com/SoR1aJn"><img src="https://i.imgur.com/SoR1aJn.png" title="source: imgur.com" /></a>

In [ ]:
# Prune
from utils.torch_utils import prune
prune(model, 0.3)

In [ ]:
!python val_prune.py --weights /content/drive/MyDrive/EAI/HRI_RA/GarbageDet/best.pt --data /content/datasets/HRI-RA-2/data.yaml --img 640 --half

## Garbage Detection ♻

In [32]:
 !python detect.py --weights /content/drive/MyDrive/EAI/HRI_RA/GarbageDet/best.pt --img 640 --conf 0.05 --source /content/drive/MyDrive/EAI/HRI_RA/GarbageDet/trial_imgs/real2.jpeg

detect: weights=['/content/drive/MyDrive/EAI/HRI_RA/GarbageDet/best.pt'], source=/content/drive/MyDrive/EAI/HRI_RA/GarbageDet/trial_imgs/real2.jpeg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.05, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-193-g485da42 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/drive/MyDrive/EAI/HRI_RA/GarbageDet/trial_imgs/real2.jpeg: 640x480 (no detections), 75.7ms
Speed: 0.7ms pre-process, 75.7ms inference, 42.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp18
